In [ ]:
# !pip install ultralytics
# !pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
import torch
from PIL import Image
import torchvision.transforms as transforms

model_data = torch.load('C:/Users/zinov/KT_YOLO_CV/best.pt', map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'), weights_only=False)
model = model_data['model']

model.eval()

im_path = "C:/Users/zinov/KT_YOLO_CV/dataset/images/4674dd1f-ashni-Wh9ZC4727e4-unsplash.jpg"

img = Image.open(im_path)

transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])
img_tensor = transform(img).unsqueeze(0)

if next(model.parameters()).dtype == torch.float16:
    img_tensor = img_tensor.half()

with torch.no_grad():
    results = model(img_tensor)

if isinstance(results, tuple):
    detections = results[0]
else:
    detections = results

In [ ]:
import torch
import pandas as pd
import numpy as np

detections = detections[0]

confidence_threshold = 0

boxes = detections[:, :4]
scores = detections[:, 4]
class_indices = detections[:, 5]

mask = scores > confidence_threshold
filtered_boxes = boxes[mask]
filtered_scores = scores[mask]
filtered_class_indices = class_indices[mask]

img_width = 640
img_height = 640

valid_boxes = []
valid_scores = []
valid_class_indices = []

def is_valid_box(xmin, ymin, xmax, ymax, img_width, img_height):
    return (0 <= xmin < xmax <= img_width) and (0 <= ymin < ymax <= img_height)

for box, score, class_index in zip(filtered_boxes, filtered_scores, filtered_class_indices):
    if is_valid_box(box[0], box[1], box[2], box[3], img_width, img_height):
        valid_boxes.append(box)
        valid_scores.append(score)
        valid_class_indices.append(class_index)

results_df = pd.DataFrame({
    'xmin': np.array(valid_boxes)[:, 0],
    'ymin': np.array(valid_boxes)[:, 1],
    'xmax': np.array(valid_boxes)[:, 2],
    'ymax': np.array(valid_boxes)[:, 3],
    'confidence': valid_scores,
    'class_index': valid_class_indices
})

print(results_df)


        xmin      ymin   xmax     ymax                             confidence  \
0  13.320312  27.09375  42.25   42.250   tensor(43.9375, dtype=torch.float16)   
1  26.765625  54.56250  91.25  104.875  tensor(108.1250, dtype=torch.float16)   

                             class_index  
0   tensor(47.9688, dtype=torch.float16)  
1  tensor(116.5625, dtype=torch.float16)  


c:\Users\zinov\KT_YOLO_CV\venv\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def clean_dataframe(df, max_class_index=2):
    df['confidence'] = df['confidence'].apply(lambda x: float(x) if isinstance(x, (torch.Tensor, np.floating)) else x)
    df['class_index'] = df['class_index'].apply(lambda x: int(float(x)) if isinstance(x, (torch.Tensor, np.floating)) else int(x))
    df[['xmin', 'xmax']] = df[['xmin', 'xmax']].apply(lambda x: [min(x), max(x)], axis=1, result_type='expand')
    df[['ymin', 'ymax']] = df[['ymin', 'ymax']].apply(lambda x: [min(x), max(x)], axis=1, result_type='expand')
    df[['xmin', 'ymin', 'xmax', 'ymax']] = df[['xmin', 'ymin', 'xmax', 'ymax']].clip(lower=0)
    df['confidence'] = df['confidence'].clip(0, 1)
    df = df[df['class_index'].between(0, max_class_index) & (df['xmax'] > df['xmin']) & (df['ymax'] > df['ymin'])]
    return df

results_df = clean_dataframe(results_df)
logger.info(f"Cleaned DataFrame:\n{results_df}")

# if results_df.empty:
#     logger.warning("Original DataFrame empty, using synthetic data")
#     results_df = pd.DataFrame({
#         'xmin': [100.0, 300.0],
#         'ymin': [150.0, 200.0],
#         'xmax': [200.0, 400.0],
#         'ymax': [250.0, 300.0],
#         'confidence': [0.85, 0.92],
#         'class_index': [0, 1]
#     })

image_path = 'C:/Users/zinov/KT_YOLO_CV/dataset/images/4674dd1f-ashni-Wh9ZC4727e4-unsplash.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (min(image.shape[1], 1000), min(image.shape[0], 1000)))
img_height, img_width = image.shape[:2]
logger.info(f"Image resized: {img_width}x{img_height}")

# Scale coordinates
def scale_coordinates(df, model_size=(640, 640)):
    df = df.copy()
    scale_x, scale_y = img_width / model_size[0], img_height / model_size[1]
    df[['xmin', 'xmax']] *= scale_x
    df[['ymin', 'ymax']] *= scale_y
    return df

results_df = scale_coordinates(results_df)

class_names = {0: "Book", 1: "Glasses", 2: "Mug"}
image_with_boxes = image.copy()
for _, row in results_df.iterrows():
    xmin, ymin, xmax, ymax = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
    confidence = row['confidence']
    class_index = int(row['class_index'])
    if xmax <= xmin or ymax <= ymin or not (0 <= confidence <= 1):
        logger.warning(f"Skipping invalid box: ({xmin}, {ymin}, {xmax}, {ymax}), conf: {confidence}")
        continue
    label = f"{class_names.get(class_index, 'Unknown')}: {confidence:.2f}"
    cv2.rectangle(image_with_boxes, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
    cv2.putText(image_with_boxes, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

cv2.imwrite("output_with_bounding_boxes.jpg", cv2.cvtColor(image_with_boxes, cv2.COLOR_RGB2BGR))
logger.info("Image saved: output_with_bounding_boxes.jpg")

dpi = 100
plt.figure(figsize=(img_width / dpi, img_height / dpi), dpi=dpi)
plt.imshow(image_with_boxes)
plt.axis('off')
plt.savefig("output_plot.png", bbox_inches='tight', pad_inches=0, dpi=dpi)
plt.close()
logger.info("Plot saved: output_plot.png")

INFO:__main__:Cleaned DataFrame:
     xmin     ymin   xmax     ymax  confidence  class_index
0  156.25  234.375  312.5  390.625        0.85            0
1  468.75  312.500  625.0  468.750        0.92            1
INFO:__main__:Image resized: 1000x1000
INFO:__main__:Image saved: output_with_bounding_boxes.jpg
INFO:__main__:Plot saved: output_plot.png
